### Les arbres de Paris
#### Contexte
Dans ce projet, on va réaliser une analyse exploratoire des données sur une dataset à propos des arbres de la ville de Paris dans le cadre du programme "Végétalisons la ville".

**On cherche à optimiser les tournées pour l'entretien des arbres.**

Pour faire cela, on va **idéntifier les caractéristiques des arbres**, et ce qui les différencie et donc comment ils sont répartis dans la ville.

### Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

### Import fichiers

In [ ]:
# import fichiers arbres
data = pd.read_csv("../p2-arbres-fr.csv", sep=";")

In [ ]:
# affichage df
display(data.head(3))

### Analyse du DF

In [ ]:
# contrôle dimensions dataframe
print("Le Dataframe contient ", data.shape[0], "lignes et ", data.shape[1], " colonnes")

In [ ]:
# contrôle types des données
data.dtypes

In [ ]:
# contrôle valeurs nulles en pourcentage dans chaque colonne différentes de 0
list_col = []
null_percentages = []

In [ ]:
for columnName in data:
    null_percentage = (data[columnName].isnull().sum() * 100 / data.shape[0])
    if null_percentage != 0:
        list_col.append(columnName)
        null_percentages.append(round(null_percentage, 2))

In [ ]:
# Création df
null_df = pd.DataFrame({
    'Colonne': list_col,
    'Pourcentages valeurs nulles': null_percentages
})

In [ ]:
# affichage
display(null_df)

On analyse maintenant les colonnes qui ont retourné des valeurs nulles
- **La colonne ``Domanialité``, ``genre``**: contient très peu des valeurs nulles, donc on vas dropper les valeurs nulles.
- Les colonnes libelle_francais, espece et autre** : pour ces colonnes on vas ajouter la catégorie "Autre" pour les arbres qui ne sont pas indiqué
- **Les colonnes ``complement adresse`` et ``numero``**: indiquent des spécifications à propos des adresses, mais vues qu' on a déjà des colonnes avec latitude et longitudes des arbres on aura pas besoin de ces colonnes.
- **La colonne ``remarquable``** contient des valeurs booléens, on va alors mettre un 0 pour les arbres qui n' ont pas d' indication.
- **La colonne ``stade_developpement``** : Pour cette colonne on vas ajouter le valeur "À mesurer"

In [ ]:
# copie du df
df = data.copy()

In [ ]:
# drop colonne numero
df.drop(columns=["numero","complement_addresse"], inplace=True)

In [ ]:
# Traitement valeur NaN dans la colonne Dominialite
df.dropna(subset = ["domanialite"], inplace=True)

In [ ]:
# Traitement valeur NaN dans la colonne genre
df.dropna(subset = ["genre"], inplace=True)

In [ ]:
# Traitement valeur NaN dans la colonne libelle francais
df['libelle_francais'] = df['libelle_francais'].fillna("autre")
df['espece'] = df['espece'].fillna("autre")
df['variete'] = df['variete'].fillna("autre")

In [ ]:
#Traitement valeur NaN dans la colonne remarquable
df['remarquable'] = df['remarquable'].fillna(0)

In [ ]:
#Traitement valeur NaN dans la colonne stade developpement
df['stade_developpement'] = df['stade_developpement'].fillna('A mesurer')

In [ ]:
# contrôle valeurs nulles
print('il y à ',df.isnull().sum().sum(),  'valeurs nulles')

Maintenant on va calculer la moyenne, la médiane et l'écart type pour certaines colonnes.
Dans ce cas, on va exclure les colonnes pour lesquelles ce calcul n'aura pas d'intérêt, en créant un subset.

In [ ]:
# Création subset pour exclure des colonnes
subsetColumns = df.loc[:, ~df.columns.isin(["id","remarquable","geo_point_2d_a","geo_point_2d_b"])]

In [ ]:
# init listes
column_names = []
means = []
medians = []
std_devs = []

In [ ]:
for columnName in subsetColumns:
    if subsetColumns[columnName].dtype != object:
        column_names.append(columnName)
        means.append(data[columnName].mean())
        medians.append(data[columnName].median())
        std_devs.append(data[columnName].std())

In [ ]:
# Creation dataframe
summary_df = pd.DataFrame({
    'colonne': column_names,
    'moyenne': means,
    'mediane': medians,
    'écart-type': std_devs
})

In [ ]:
display(summary_df)

### Récherche outliers
Maintenant on va identifier les outliers en utilisant deux méthodes:
1. On vas afficher les valeurs max et min pour la hauteur et la circonférence pour observer le comportement des données
2. On vas utiliser la méthode des quantiles pour détecter les outliers

In [ ]:
# creation subset pour rendre plus lisibles les résultats
subset_df = df[['id', 'libelle_francais', 'circonference_cm', 'hauteur_m']].copy()

In [ ]:
# création variables pour stocker valeurs min et max de la circonference
min_circ = subset_df.sort_values(by="circonference_cm").head(5)
max_circ = subset_df.sort_values(by="circonference_cm", ascending=False).head(5)

In [ ]:
# affichage df
display(min_circ)

Présence de certains arbres avec une circonférence qui équivaut à 0 cm

In [ ]:
# affichage df
display(max_circ)

Présence de certains arbres avec une circonférence qui équivaut à des valeurs trop hautes pour être considéré comme possibles.

In [ ]:
# création variables pour stocker valeurs min et max de la circonference
min_haut = subset_df.sort_values(by="hauteur_m").head(5)
max_haut = subset_df.sort_values(by="hauteur_m", ascending=False).head(5)

In [ ]:
# affichage df
display(max_haut)

In [ ]:
# affichage df
display(min_haut)

 Création d'une fonction pour mettre en pratique la méthode interquartile pour détecter des outliers et donc on vas procéder avec la création d'une autre data frame ou on vas enlever ces dernier

In [ ]:
# méthode pour calculer IQR
def outlier_IQR (colonne):
    # calcul Quartile 1 et 3
    Q1,Q3 = np.percentile(colonne , [25,75])
    # calcul intervalle interquartile
    IQR = Q3 - Q1
    # limit superieure
    ls = Q3+1.5*IQR
    # limit inferieure
    li = Q1-1.5*IQR
    outliers = colonne[(colonne > ls) | (colonne < li)]
    return outliers

In [ ]:
# Création outlier
outlier_circ = outlier_IQR (df["circonference_cm"])
outlier_haut = outlier_IQR (df["hauteur_m"])

In [ ]:
# Création dataframe temp
df1 = df.copy()

In [ ]:
# Supprimer outliers
df1 = df1[~df1["circonference_cm"].isin(outlier_circ)]
df1 = df1[~df1["hauteur_m"].isin(outlier_haut)]

### Quelles sont les caracteristiques arbres?

In [ ]:
# affichage graphique "circonference_cm"
sns.boxplot(data=df1, x="circonference_cm",
            palette='Paired')
plt.title("Boxplot Circonference en cm")
plt.show()

On voit que la concentration des données est entre 45 et 110 cm de circonférence.
Le boxplot se concentre vers le bas, et on observe des arbres qui peuvent arriver aussi a 200 cm de circonférence mais ils sont plus rares.

In [ ]:
# affichage graphique "hauter_m"
sns.boxplot(df1, x="hauteur_m",
            palette='Paired')
            
plt.title("Boxplot hauteur en metres")

plt.show()

On observe que les arbres se concentrent en général entre 4 et 13 mètres.
On voit que le boxplot se concentre plutôt vers le bas, mais il y a des arbres qui sont plus rares, qui peuvent arriver jusqu'à 22 mètres de hauteur.

Maintenant on va observer la dispersion des données dans une scatterplot, en mettant en évidence le stade de développement des arbres.
Tout d'abord on va changer les noms des étiquettes des arbres pour qu' ils soient plus clairs, et donc on procède avec la représentation graphique.

In [ ]:
# changer valeurs dans le dataframe
df1 = df1.replace(['A'], 'Adulte')
df1 = df1.replace(['J'], 'Jeune')
df1 = df1.replace(['JA'], 'Jeune Adulte')
df1 = df1.replace(['M'], 'Mature')
    

In [ ]:
# création graphique et affichage
sns.scatterplot(data=df1, 
                x='circonference_cm', 
                y='hauteur_m', 
                hue='stade_developpement')
plt.legend(
    bbox_to_anchor=(1.05, 1), 
    loc='upper left', 
    borderaxespad=0)
plt.title(
    "Graphique à dispersion pour comparer hauteur et circonference entre stades de développement")
plt.xlabel("Circonference en CM")
plt.ylabel("Hauteur en metres")
plt.show()

On peut identifier diverses caractéristiques.
- Les arbres jeunes : tends a ce concentrer dans la part plus bas du graphique en générale et donc on voit qu' ils sont plus petit et avec un circonference plus petit
- Les arbres jeunes adulte: sont plutôt concentré vers la gauche du graphique, mais il commencent à varier un peu plus en hauteur probablement en relation â l'espèce d'arbres
- Les arbres adulte : sont les arbres les plus répandus dans le graphique et nous donnent un idée de la variation entre circonférence et hauteur, en relation à l' espèce très variées des arbres
- Les arbres mature : sont les arbres que se concentrent plus vers la droite, et ici, on voit qu' ils ont une circonférence plus importante et aussi ils sont les arbres tendanciellement plus hautes.

Maintenant on vas essayer des confirmer ces observation en observant des plus prés le comportement des données avec des boxplot divisé par stade de développement

In [ ]:
# création graphique et affichage
sns.boxplot(data=df1, 
            x="stade_developpement",
            y="circonference_cm",
            order=['Jeune','Jeune Adulte','Adulte','Mature'],
            hue='stade_developpement',
            palette='Paired'),
            
plt.title(
    "Comparaison des boxplots des circonferences en cm par stade de développement")
plt.xlabel("Stade de développement")
plt.ylabel("Circonference en cm")
plt.show()

In [ ]:
# création graphique et affichage
bplot = sns.boxplot(data=df1, x="stade_developpement", y="hauteur_m", order=['Jeune','Jeune Adulte','Adulte','Mature'],hue='stade_developpement', palette='Paired')
plt.title("Comparaison des boxplots des hauteurs en metres par stade de développement")
plt.xlabel("Stade de développement")
plt.ylabel("Hauter en metres")
plt.show()

### Quelle typologie des arbres?
Quels sont les typologies des arbres les plus répandues à Paris selon le dataset mis à disposition ? 

In [ ]:
# counts des valeurs dans le dataframe
counts = df1["libelle_francais"].value_counts()

In [ ]:
# Création graphique
fig = px.pie(counts, names=counts.index, values=counts.values, title="Camembert typologie arbres")
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.show()

Le platane est l'arbre plus répandu avec un 20,3% ($9423$) du total, et puis suivent le Marronnier avec 12,3% ($23953$) et le Tilleul avec 10,9% ($21152$)

### Ou se trouvent les arbres?
Affichage des numéros des arbres par arrondissements

In [ ]:
# création barplot pour déterminer quelles sont les arrondissemnt
df1["arrondissement"].value_counts().plot(kind = 'barh')
plt.title("Distribution arbres par arrondissement")
plt.ylabel("Arrondissements")
plt.xlabel("Arbres")
plt.show()

On voit que les arrondissements avec la concentration majeure des arbres sont le 15eme, le 13eme et le 16eme

In [ ]:
# Affichage des arbres dans un carte par stade de dévelopement
fig = px.scatter_mapbox(df1,
                        lat='geo_point_2d_a', lon='geo_point_2d_b',
                        hover_data=['circonference_cm', 'hauteur_m', 'arrondissement'],
                        color = 'stade_developpement',
                        zoom=11,
                        mapbox_style="stamen-terrain",
                        title="carte des arbres par stade de developpement",
                        width=800,
                        height=500,
                        )
fig.show()

Dans cette carte on peut observer la localisation des arbres par leur stade de développement, et on peut surtout localiser les arbres plus matures qui auront besoin de manutention, et donc descendre dans la hiérarchie pour afficher les autres arbres au fur et mesure des besoins.

In [ ]:
# création barplot pour déterminer quelles sont les arrondissemnt
df1["stade_developpement"].value_counts().plot(kind = 'barh')
plt.ylabel("Stade développement")
plt.xlabel("Arbres")
plt.title("Numero des arbres par stade de développement")
plt.show()

On observe la présence de beaucoup d’arbres pour lesquels le stade de développement reste à mesurer, et si on pourra le faire nous permettra de mieux cibler les arbres par âge, et donc mieux optimiser leur maintenance

In [ ]:
# Affichage des arbres dans une carte par genre
fig = px.scatter_mapbox(df1,
                        lat='geo_point_2d_a', lon='geo_point_2d_b',
                        hover_data=['circonference_cm', 'hauteur_m', 'arrondissement'],
                        color = 'libelle_francais',
                        zoom=11,
                        mapbox_style="stamen-terrain",
                        title="carte des arbres par typologie d'arbre",
                        width=800,
                        height=500,
                        )
fig.show()

Avec une légende des typologies des entretiens en fonction du stade de développement et de la typologie des arbres, on pourra croiser les deux graphiques créés pour optimiser les routes suivies pour effectuer les opérations de manutention.